# Wikification

In [104]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

from PyPDF2 import PdfReader
import os
import pandas as pd
import numpy as np

import sys
sys.path.append('../src/')
import wikifier

Number of processors:  8


In [57]:
path = "../dat/books/all_books/"
data = pd.DataFrame(columns = ["file","text"])

book_files = os.listdir(path)

for j, book in enumerate(book_files):
    reader = PdfReader(path + book)
    print("Extracting " + book)
    text = ""
    k = 0
    for page in reader.pages:
        k+=1
        if k%50==0: print("Extract page" + str(k))
        text += str(page.extract_text()) + " "

    data = pd.concat([data, pd.DataFrame.from_records([{'file': book, 'text': text}])])
data.reset_index()

Extracting CollegeAlgCoreq-WEB.pdf
Extract page50
Extract page100
Extract page150
Extract page200
Extract page250
Extract page300
Extract page350
Extract page400
Extract page450
Extract page500
Extract page550
Extract page600
Extract page650
Extract page700
Extract page750
Extract page800
Extract page850
Extract page900
Extract page950
Extract page1000
Extract page1050
Extract page1100
Extract page1150
Extract page1200
Extract page1250
Extracting Linear algebra — Strang.pdf


unknown widths : 
[0, IndirectObject(24944, 0, 140256693795424)]
unknown widths : 
[0, IndirectObject(24947, 0, 140256693795424)]
unknown widths : 
[0, IndirectObject(24950, 0, 140256693795424)]
unknown widths : 
[0, IndirectObject(24953, 0, 140256693795424)]
unknown widths : 
[0, IndirectObject(24956, 0, 140256693795424)]
unknown widths : 
[0, IndirectObject(24959, 0, 140256693795424)]
unknown widths : 
[0, IndirectObject(24962, 0, 140256693795424)]
unknown widths : 
[0, IndirectObject(24965, 0, 140256693795424)]
unknown widths : 
[0, IndirectObject(24968, 0, 140256693795424)]
unknown widths : 
[0, IndirectObject(24971, 0, 140256693795424)]
unknown widths : 
[0, IndirectObject(24974, 0, 140256693795424)]
unknown widths : 
[0, IndirectObject(25184, 0, 140256693795424)]
unknown widths : 
[0, IndirectObject(25187, 0, 140256693795424)]
unknown widths : 
[0, IndirectObject(25190, 0, 140256693795424)]
unknown widths : 
[0, IndirectObject(25193, 0, 140256693795424)]
unknown widths : 
[0, Ind

Extract page50
Extract page100
Extract page150
Extract page200
Extract page250
Extract page300
Extract page350
Extract page400
Extract page450
Extract page500
Extract page550
Extracting linear-Cherey, Denton.pdf
Extract page50
Extract page100
Extract page150
Extract page200
Extract page250
Extract page300
Extract page350
Extract page400
Extracting ElementaryAlgebra2e-WEB_EjIP4sI.pdf
Extract page50
Extract page100
Extract page150
Extract page200
Extract page250
Extract page300
Extract page350
Extract page400
Extract page450
Extract page500
Extract page550
Extract page600
Extract page650
Extract page700
Extract page750
Extract page800
Extract page850
Extract page900
Extract page950
Extract page1000
Extract page1050
Extract page1100
Extract page1150
Extract page1200
Extract page1250
Extract page1300
Extracting interactive_textbook.pdf
Extract page50
Extract page100
Extract page150
Extract page200
Extract page250
Extract page300
Extract page350
Extract page400
Extract page450
Extracting te

unknown widths : 
[0, IndirectObject(4281, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(4285, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(4289, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(4293, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3232, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3236, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3187, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3190, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(4281, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3242, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3246, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3232, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3232, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3250, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3242, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(324

Extract page50


...

Extract page350


unknown widths : 
[0, IndirectObject(3274, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3471, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3803, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(4131, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(4139, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3395, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3771, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3913, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(4004, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3351, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3242, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3270, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3300, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3675, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3232, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(328

Extract page400


unknown widths : 
[0, IndirectObject(3232, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3514, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3236, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3731, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3300, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3327, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3331, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(4123, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3282, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3395, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3274, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3351, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3258, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3232, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(3327, 0, 140256273013184)]
unknown widths : 
[0, IndirectObject(328

Extracting Nicholson-OpenLAWA-2019A.pdf
Extract page50
Extract page100
Extract page150
Extract page200
Extract page250
Extract page300
Extract page350
Extract page400
Extract page450
Extract page500
Extract page550
Extract page600
Extract page650
Extracting Linear algebra done right — Axler.pdf
Extract page50
Extract page100
Extract page150
Extract page200
Extract page250
Extract page300
Extract page350
Extracting Math1410_print.pdf
Extract page50
Extract page100
Extract page150
Extract page200
Extract page250
Extract page300
Extract page350
Extract page400
Extracting Hefferon_LinAlgebra.pdf
Extract page50
Extract page100
Extract page150
Extract page200
Extract page250
Extract page300
Extract page350
Extract page400
Extract page450
Extract page500
Extracting Kuttler-LinearAlgebra-AFirstCourse-2017A.pdf
Extract page50
Extract page100
Extract page150
Extract page200
Extract page250
Extract page300
Extract page350
Extract page400
Extract page450
Extract page500
Extract page550
Extract pag

In [116]:
data.text.apply(len)

# exclude Math14, Strang

file
CollegeAlgCoreq-WEB.pdf                         1742146
Linear algebra — Strang.pdf                     1151894
linear-Cherey, Denton.pdf                        485450
ElementaryAlgebra2e-WEB_EjIP4sI.pdf             1628431
interactive_textbook.pdf                         578111
textbook_Hoffman_Kunze.pdf                       790385
Nicholson-OpenLAWA-2019A.pdf                    1402815
Linear algebra done right — Axler.pdf            520380
Math1410_print.pdf                               740115
Hefferon_LinAlgebra.pdf                          876866
Kuttler-LinearAlgebra-AFirstCourse-2017A.pdf     929580
Beezer_First_Course.pdf                         1077274
Name: text, dtype: int64

In [131]:
data.text[8]

'MathƈƋƈƇElementaryLinearAlgebra\nSpring ƈƆƈƆ Ediࢡon ,UniversityofLethbridge\n(ThisediࢢonisessenࢢallyunchangedfromtheSpringƉƇƈқediࢢon.)\nEditor:SeanFitzpatrick\nDepartment of Mathemaࢡcs and Computer Science\nUniversity of Lethbridge\nContribuࢡng Textbooks\nPrecalculus\nVersion ⌊\x19⌋= 3\nCarl Sࢡtz and Jeﬀ Zeager\nwww.sࢢtz-zeager.com\nFundamentals of Matrix Algebra\nThirdEdiࢢon,VersionƊ.ƈƈƈƇ\nGregory Hartman\nwww.vmi.edu\nAPEX Calculus\nVersionƊ.Ƈ\nGregory Hartman et al\napexcalculus.com ChaptersƈandƉCopyright©ƉƇƈƊCarlSࢢtzandJeﬀZeager\nLicensed under the Creaࢢve Commons A\x82ribuࢢon-\nNoncommercial-ShareAlikeƊ.ƇUnportedPublicLicense\nChapterƊCopyright©ƉƇƈƌGregoryHartman\nLicensed under the Creaࢢve Commons A\x82ribuࢢon-\nNoncommercialƋ.ƇInternaࢢonalPublicLicense\nChaptersƋ-қCopyright©ƉƇƈƈGregoryHartman,exceptas\nnotedbelow.\nLicensed under the Creaࢢve Commons A\x82ribuࢢon-\nNoncommercialƊ.ƇUnitedStatesLicense\nThisversionofthetextwasassembledandeditedbySeanFitzpatrick,Uni-\nversityofLeth

In [121]:
batch_size = 5000
num_threads = mp.cpu_count()

all_annotations = []
for i in range(2,3):
    text = data.iloc[i].text

    pool = mp.Pool(num_threads)
    print("Length: " + str(len(text)))
    annotations = [pool.apply(wikifier.get_annotations, args=[text[i*batch_size:(i+1)*batch_size], i*batch_size])
        for i in range(int(np.floor(len(text)/batch_size)))]
    pool.close()
    all_annotations = sum(annotations, [])

Length: 485450
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
<class 'Exception'>
65000
<class 'Exception'>
70000
<class 'Exception'>
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
345000
350000
355000
<class 'Exception'>
360000
<class 'Exception'>
365000
370000
375000
380000
385000
390000
<class 'Exception'>
395000
400000
405000
410000
415000
420000
425000
430000
435000
440000
445000
450000
455000
460000
465000
470000
475000
480000


In [113]:
annos = [{
    "title": a["title"],
    "supports": [{'chFrom': s['chFrom'], 'chTo': s['chTo']} for s in a["support"] if s['pageRank']>0.001]
    } for a in all_annotations]
[a for a in annos if len(a['supports'])]

[{'title': 'St. Louis Community College',
  'supports': [{'chFrom': 103, 'chTo': 129}]},
 {'title': 'Community college', 'supports': [{'chFrom': 113, 'chTo': 129}]},
 {'title': 'Arizona State University',
  'supports': [{'chFrom': 145, 'chTo': 168}]},
 {'title': 'Rice University',
  'supports': [{'chFrom': 194, 'chTo': 208},
   {'chFrom': 399, 'chTo': 413},
   {'chFrom': 1690, 'chTo': 1704},
   {'chFrom': 1716, 'chTo': 1730},
   {'chFrom': 1843, 'chTo': 1857},
   {'chFrom': 2301, 'chTo': 2315},
   {'chFrom': 2521, 'chTo': 2535},
   {'chFrom': 2629, 'chTo': 2643}]},
 {'title': 'Houston', 'supports': [{'chFrom': 234, 'chTo': 247}]},
 {'title': 'HTML',
  'supports': [{'chFrom': 760, 'chTo': 763}, {'chFrom': 1165, 'chTo': 1168}]},
 {'title': 'Peer review', 'supports': [{'chFrom': 2098, 'chTo': 2110}]},
 {'title': 'Advanced Placement', 'supports': [{'chFrom': 2168, 'chTo': 2185}]},
 {'title': 'Research university',
  'supports': [{'chFrom': 2551, 'chTo': 2569}]},
 {'title': 'Arnold Ventures

In [129]:
len(all_annotations)
import pickle
file = open("../dat/annotations_Denton.pkl", 'wb')
pickle.dump(all_annotations, file)
file.close()